## Installation

```
$ pip install requests mapboxgl supermercado
```

In [ ]:
%pylab inline

import os
import json
import random
import requests
import datetime
from io import BytesIO
import urllib.parse

from supermercado.burntiles import tile_extrema

from mapboxgl.utils import *
from mapboxgl.viz import *

token = os.environ["MAPBOX_ACCESS_TOKEN"]

endpoint = "https://landsatlive.live"

### Create Stack of NDVI values

In [ ]:
# STAC QUERY (sat-api)
bounds = [24.98291015625, 36.01356058518153, 40.31982421875, 42.342305278572816] # Turkey

# Cloud Filters
min_cloud = 0
max_cloud = 1

# Date Filters
date_min="2013-01-01"
date_max="2019-12-01"
start = datetime.datetime.strptime(date_min, "%Y-%m-%d").strftime("%Y-%m-%dT00:00:00Z")
end = datetime.datetime.strptime(date_max, "%Y-%m-%d").strftime("%Y-%m-%dT23:59:59Z")

query = {
    "bbox": bounds,
    "time": f"{start}/{end}",
    "query": {
        "eo:sun_elevation": {"gt": 0},
        "landsat:tier": {"eq": "T1"},
        "collection": {"eq": "landsat-8-l1"},
        "eo:cloud_cover": {"gte": min_cloud, "lt": max_cloud}
    },
}

# We post the query to the mosaic endpoint
# `optimized_selection: True` is an option to force only one PATH-ROW scene per tile
# Because we will use `pixel_selection:"first"` this will minimized the number of image fetched
r = requests.post(
    f"{endpoint}/mosaic/create", 
    json=query,
    params = {
        "optimized_selection": False,
        "maximum_items_per_tile": 0, # We keep all intersecting scenes
        "seasons": "spring" # this should be a "," separated list of seasons
    },
)
results = r.json()

In [ ]:
mosaicid = results["name"]
geojson = requests.get(f"{endpoint}/mosaic/{mosaicid}/geojson").json()

print("Number of scenes per Quadkey")
print([(x["properties"]["quadkey"], len(x["properties"]["files"])) for x in geojson["features"]])

In [ ]:
viz = LinestringViz(geojson,
    access_token=token,
    line_width_default=4,
    center=((bounds[0] + bounds[2]) / 2, (bounds[1] + bounds[3]) / 2),
    zoom=4,
)
viz.show()

In [ ]:
# Get Mercator tiles covering the AOI
zoom = results["maxzoom"]
extrema = tile_extrema(bounds, zoom)
tiles = []
for x in range(extrema["x"]["min"], extrema["x"]["max"]):
    for y in range(extrema["y"]["min"], extrema["y"]["max"]):
        tiles.append(f"{zoom}-{x}-{y}")
        
tile = tiles[random.randint(0, len(tiles))]
z, x, y = list(map(int, tile.split("-")))

In [ ]:
# Create tile url
# pixel_selection="all" will create a stack of all scenes available
query_params = dict(
    expr="(b5-b4)/(b5+b4)",
    pixel_selection="all"
)

# .npy means the endpoint should return a numpy array/list
tile_url = f"{endpoint}/tiles/{mosaicid}/{z}/{x}/{y}.npy?" + urllib.parse.urlencode(query_params)
r = requests.get(tile_url)
arr = numpy.load(BytesIO(r.content), allow_pickle=True)

# the endpoint should return a list of tiles and masks
tiles, masks = arr
print(len(tiles))
print(len(masks))

In [ ]:
imshow(tiles[0][0])

In [ ]:
imshow(tiles[1][0])